# 3.7 学習と検証の実施

- 本ファイルでは、PSPNetの学習と検証の実施を行います。AWSのGPUマシンで計算します。
- p2.xlargeで●時間かかります。


# 学習目標

1.	PSPNetの学習と検証を実装できるようになる
2.	セマンティックセグメンテーションのファインチューニングを理解する


# 事前準備

- 本書に従い学習済みモデルのファイル「pspnet50_ADE20K.pth」をダウンロードし、フォルダ「weights」に用意します。

In [1]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# 初期設定
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DataLoader作成

In [3]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

# ファイルパスリスト作成
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

# Dataset作成
# (RGB)の色の平均値と標準偏差
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

# DataLoader作成
batch_size = 8

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# ネットワークモデル作成

In [4]:
from utils.pspnet import PSPNet

# ファインチューニングでPSPNetを作成
# ADE20Kデータセットの学習済みモデルを使用、ADE20Kはクラス数が150です
net = PSPNet(n_classes=150)

# ADE20K学習済みパラメータをロード
state_dict = torch.load("./weights/pspnet50_ADE20K.pth")
net.load_state_dict(state_dict)

# 分類用の畳み込み層を、出力数21のものにつけかえる
n_classes = 21
net.decode_feature.classification = nn.Conv2d(
    in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

net.aux.classification = nn.Conv2d(
    in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

# 付け替えた畳み込み層を初期化する。活性化関数がシグモイド関数なのでXavierを使用する。


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)


net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)


print('ネットワーク設定完了：学習済みの重みをロードしました')


ネットワーク設定完了：学習済みの重みをロードしました


In [5]:
net

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNeckPSP(
      (cb

# 損失関数を定義

In [6]:
# 損失関数の設定
class PSPLoss(nn.Module):
    """PSPNetの損失関数のクラスです。"""

    def __init__(self, aux_weight=0.4):
        super(PSPLoss, self).__init__()
        self.aux_weight = aux_weight  # aux_lossの重み

    def forward(self, outputs, targets):
        """
        損失関数の計算。

        Parameters
        ----------
        outputs : PSPNetの出力(tuple)
            (output=torch.Size([num_batch, 21, 475, 475]), output_aux=torch.Size([num_batch, 21, 475, 475]))。

        targets : [num_batch, 475, 4755]
            正解のアノテーション情報

        Returns
        -------
        loss : テンソル
            損失の値
        """

        loss = F.cross_entropy(outputs[0], targets, reduction='mean')
        loss_aux = F.cross_entropy(outputs[1], targets, reduction='mean')

        return loss+self.aux_weight*loss_aux


criterion = PSPLoss(aux_weight=0.4)


# 最適化手法を設定

In [7]:
# ファインチューニングなので、学習率は小さく
optimizer = optim.SGD([
    {'params': net.feature_conv.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_2.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_2.parameters(), 'lr': 1e-3},
    {'params': net.pyramid_pooling.parameters(), 'lr': 1e-3},
    {'params': net.decode_feature.parameters(), 'lr': 1e-2},
    {'params': net.aux.parameters(), 'lr': 1e-2},
], momentum=0.9, weight_decay=0.0001)


# スケジューラーの設定
def lambda_epoch(epoch):
    max_epoch = 30
    return math.pow((1-epoch/max_epoch), 0.9)


scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)


# 学習・検証を実施する

In [8]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # 画像の枚数
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    num_val_imgs = len(dataloaders_dict["val"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # イタレーションカウンタをセット
    iteration = 1
    logs = []

    # multiple minibatch
    batch_multiplier = 3

    # epochのループ
    for epoch in range(num_epochs):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                scheduler.step()  # 最適化schedulerの更新
                optimizer.zero_grad()
                print('（train）')

            else:
                if((epoch+1) % 5 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('（val）')
                else:
                    # 検証は5回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            count = 0  # multiple minibatch
            for imges, anno_class_imges in dataloaders_dict[phase]:
                # ミニバッチがサイズが1だと、バッチノーマライゼーションでエラーになるのでさける
                if imges.size()[0] == 1:
                    continue

                # GPUが使えるならGPUにデータを送る
                imges = imges.to(device)
                anno_class_imges = anno_class_imges.to(device)

                if (phase == 'train') and (count == 0):
                    optimizer.step()
                    optimizer.zero_grad()
                    count = batch_multiplier

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(imges)
                    loss = criterion(
                        outputs, anno_class_imges.long()) / batch_multiplier

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算
                        count -= 1  # multiple minibatch

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イタレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size*batch_multiplier, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item() * batch_multiplier
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item() * batch_multiplier

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1, 'train_loss': epoch_train_loss /
                     num_train_imgs, 'val_loss': epoch_val_loss/num_val_imgs}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

    # 最後のネットワークを保存する
    torch.save(net.state_dict(), 'weights/pspnet50_' +
               str(epoch+1) + '.pth')


In [9]:
# 学習・検証を実行する
num_epochs = 30
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-------------
Epoch 1/30
-------------
（train）
イタレーション 10 || Loss: 0.3809 || 10iter: 85.7882 sec.
イタレーション 20 || Loss: 0.2182 || 10iter: 54.1508 sec.
イタレーション 30 || Loss: 0.1513 || 10iter: 53.7752 sec.
イタレーション 40 || Loss: 0.1653 || 10iter: 54.4371 sec.
イタレーション 50 || Loss: 0.0889 || 10iter: 53.9188 sec.
イタレーション 60 || Loss: 0.0728 || 10iter: 54.0658 sec.
イタレーション 70 || Loss: 0.1163 || 10iter: 53.6033 sec.
イタレーション 80 || Loss: 0.1338 || 10iter: 53.7854 sec.
イタレーション 90 || Loss: 0.2167 || 10iter: 54.4044 sec.
イタレーション 100 || Loss: 0.0897 || 10iter: 54.1622 sec.
イタレーション 110 || Loss: 0.1403 || 10iter: 53.9190 sec.
イタレーション 120 || Loss: 0.0668 || 10iter: 54.2026 sec.
イタレーション 130 || Loss: 0.1241 || 10iter: 53.9137 sec.
イタレーション 140 || Loss: 0.1469 || 10iter: 53.6589 sec.
イタレーション 150 || Loss: 0.0788 || 10iter: 54.2437 sec.
イタレーション 160 || Loss: 0.1428 || 10iter: 54.2170 sec.
イタレーション 170 || Loss: 0.2082 || 10iter: 54.1832 sec.
イタレーション 180 || Loss: 0.1429 || 10iter: 54.1187 sec.
-----------

イタレーション 1380 || Loss: 0.0702 || 10iter: 54.0981 sec.
イタレーション 1390 || Loss: 0.0781 || 10iter: 54.1621 sec.
イタレーション 1400 || Loss: 0.0591 || 10iter: 54.0377 sec.
イタレーション 1410 || Loss: 0.0693 || 10iter: 54.1390 sec.
イタレーション 1420 || Loss: 0.0748 || 10iter: 54.5626 sec.
イタレーション 1430 || Loss: 0.1073 || 10iter: 54.2815 sec.
イタレーション 1440 || Loss: 0.0624 || 10iter: 54.0759 sec.
イタレーション 1450 || Loss: 0.0553 || 10iter: 54.1870 sec.
イタレーション 1460 || Loss: 0.0365 || 10iter: 54.1660 sec.
-------------
epoch 8 || Epoch_TRAIN_Loss:0.0569 ||Epoch_VAL_Loss:0.0000
timer:  1094.9084 sec.
-------------
Epoch 9/30
-------------
（train）
イタレーション 1470 || Loss: 0.0465 || 10iter: 30.6023 sec.
イタレーション 1480 || Loss: 0.0206 || 10iter: 54.9038 sec.
イタレーション 1490 || Loss: 0.0392 || 10iter: 54.1856 sec.
イタレーション 1500 || Loss: 0.0461 || 10iter: 54.1716 sec.
イタレーション 1510 || Loss: 0.0357 || 10iter: 54.2890 sec.
イタレーション 1520 || Loss: 0.0358 || 10iter: 54.0945 sec.
イタレーション 1530 || Loss: 0.1017 || 10iter: 54.1214 sec.
イタレーション 1

イタレーション 2740 || Loss: 0.0266 || 10iter: 54.1817 sec.
-------------
（val）
-------------
epoch 15 || Epoch_TRAIN_Loss:0.0454 ||Epoch_VAL_Loss:0.0712
timer:  1465.1200 sec.
-------------
Epoch 16/30
-------------
（train）
イタレーション 2750 || Loss: 0.0719 || 10iter: 24.3353 sec.
イタレーション 2760 || Loss: 0.0291 || 10iter: 54.0146 sec.
イタレーション 2770 || Loss: 0.0289 || 10iter: 53.9887 sec.
イタレーション 2780 || Loss: 0.0385 || 10iter: 54.1129 sec.
イタレーション 2790 || Loss: 0.0379 || 10iter: 54.0923 sec.
イタレーション 2800 || Loss: 0.0459 || 10iter: 54.0815 sec.
イタレーション 2810 || Loss: 0.0315 || 10iter: 54.1502 sec.
イタレーション 2820 || Loss: 0.0307 || 10iter: 54.1893 sec.
イタレーション 2830 || Loss: 0.0475 || 10iter: 53.9634 sec.
イタレーション 2840 || Loss: 0.0258 || 10iter: 54.0413 sec.
イタレーション 2850 || Loss: 0.0435 || 10iter: 54.2436 sec.
イタレーション 2860 || Loss: 0.0519 || 10iter: 54.1655 sec.
イタレーション 2870 || Loss: 0.0288 || 10iter: 54.1621 sec.
イタレーション 2880 || Loss: 0.0290 || 10iter: 53.9877 sec.
イタレーション 2890 || Loss: 0.0364 || 10iter: 

イタレーション 4060 || Loss: 0.0293 || 10iter: 54.2066 sec.
イタレーション 4070 || Loss: 0.0239 || 10iter: 54.2813 sec.
イタレーション 4080 || Loss: 0.0662 || 10iter: 54.3449 sec.
イタレーション 4090 || Loss: 0.0234 || 10iter: 54.3371 sec.
イタレーション 4100 || Loss: 0.0332 || 10iter: 54.2052 sec.
イタレーション 4110 || Loss: 0.0284 || 10iter: 54.2600 sec.
イタレーション 4120 || Loss: 0.0401 || 10iter: 54.4572 sec.
イタレーション 4130 || Loss: 0.0499 || 10iter: 54.3100 sec.
イタレーション 4140 || Loss: 0.0366 || 10iter: 54.3269 sec.
イタレーション 4150 || Loss: 0.0455 || 10iter: 54.5363 sec.
イタレーション 4160 || Loss: 0.0442 || 10iter: 54.2133 sec.
イタレーション 4170 || Loss: 0.0750 || 10iter: 54.3995 sec.
イタレーション 4180 || Loss: 0.0446 || 10iter: 54.3855 sec.
イタレーション 4190 || Loss: 0.0391 || 10iter: 54.1874 sec.
イタレーション 4200 || Loss: 0.0412 || 10iter: 54.2442 sec.
-------------
epoch 23 || Epoch_TRAIN_Loss:0.0417 ||Epoch_VAL_Loss:0.0000
timer:  1097.0536 sec.
-------------
Epoch 24/30
-------------
（train）
イタレーション 4210 || Loss: 0.0190 || 10iter: 0.3346 sec.
イタレーション 

イタレーション 5420 || Loss: 0.0186 || 10iter: 53.7726 sec.
イタレーション 5430 || Loss: 0.0413 || 10iter: 53.9767 sec.
イタレーション 5440 || Loss: 0.0484 || 10iter: 54.0522 sec.
イタレーション 5450 || Loss: 0.0491 || 10iter: 54.0189 sec.
イタレーション 5460 || Loss: 0.0440 || 10iter: 54.2314 sec.
イタレーション 5470 || Loss: 0.0187 || 10iter: 54.1328 sec.
イタレーション 5480 || Loss: 0.0654 || 10iter: 54.1384 sec.
イタレーション 5490 || Loss: 0.0625 || 10iter: 54.1472 sec.
-------------
（val）
-------------
epoch 30 || Epoch_TRAIN_Loss:0.0391 ||Epoch_VAL_Loss:0.0698
timer:  1463.1491 sec.


以上